In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../examples/optimized/data/simple_two_bumps_3/flat_opt.msh')
driver=l.centralJoint()

mat = elastic_rods.RodMaterial('Rectangle', 2000, 0.3, [15.3,8.1], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('../examples/optimized/data/simple_two_bumps_3/design_parameters.txt'))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
# fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
open_linkage(l, driver, np.deg2rad(76) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);
# open_linkage(l, driver, np.deg2rad(15) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

In [ ]:
# Output fabrication data
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_meshID_1d1c1dc9-4638-474e-8f7f-6b95998b8a32.txt', zeroBasedIndexing=True)
np.savetxt('restlen_meshID_1d1c1dc9-4638-474e-8f7f-6b95998b8a32.txt',l.getPerSegmentRestLength())

In [ ]:
# Output rendering data (rest configuration)
writeRodSegments(l,'rod_segments_2bumps_rest.txt', zeroBasedIndexing=True)
np.savetxt('restlen_2bumps_rest.txt',l.getPerSegmentRestLength())
np.savetxt('normals_2bumps_rest.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_2bumps_rest.msh')

In [ ]:
import compute_vibrational_modes
fixedVarsWithoutActuator = fixedVars[:-1]
# lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=fixedVarsWithoutActuator, mtype=compute_vibrational_modes.MassMatrixType.FULL)
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-2)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=5.0)
mview.show()

In [ ]:
# Output rendering data (deployed configuration)
writeRodSegments(l,'rod_segments_2bumps_95.txt', zeroBasedIndexing=True)
np.savetxt('restlen_2bumps_95.txt',l.getPerSegmentRestLength())
np.savetxt('normals_2bumps_95.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_2bumps_95.msh')